##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = tf.keras.utils.get_file('caiqueocoelho.txt', 'https://raw.githubusercontent.com/CaiqueCoelho/tensorflow-text-generation/main/new_caique_new_tweets.txt')

188416/186297 [==============================] - 0s 0us/step


### Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 182119 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Saudades demais gatão
Saudades dona Ercilha
Meus pêsames Lu, não existem palavras para confortar neste momento, mas se precisar conversar, estou por aqui
_ Autoestima abaixou na pandemia
Parabéns Erika!
Agora já to me achando hahahaha obrigado por fa


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

125 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')
chars

<tf.Tensor: shape=(182119,), dtype=string, numpy=array([b'S', b'a', b'u', ..., b'r', b'a', b'\n'], dtype=object)>

Now create the `preprocessing.StringLookup` layer:

In [7]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts form tokens to character IDs:

In [8]:
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(182119,), dtype=int64, numpy=array([50, 61, 81, ..., 78, 61,  1])>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [9]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(182119,), dtype=string, numpy=array([b'S', b'a', b'u', ..., b'r', b'a', b'\n'], dtype=object)>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

b'Saudades demais gat\xc3\xa3o\nSaudades dona Ercilha\nMeus p\xc3\xaasames Lu, n\xc3\xa3o existem palavras para confortar neste momento, mas se precisar conversar, estou por aqui\n_ Autoestima abaixou na pandemia\nParab\xc3\xa9ns Erika!\nAgora j\xc3\xa1 to me achando hahahaha obrigado por fazer meu dia Natinha\nEu queria ter pelo menos 1% do charme do \nAcabamos de abrir vagas para QA aqui no QuintoAndar, que tal vir revolucionar o mercado imobili\xc3\xa1rio conosco? Estamos com grandes desafios na \xc3\xa1rea, com muito espa\xc3\xa7o para inova\xc3\xa7\xc3\xa3o e crescimento. As vagas s\xc3\xa3o para todos os n\xc3\xadveis. Ahh\xe2\x80\xa6 \n_ contratado\nkkkkkkk tipo isso\nEssas propagandas no meio do jogo com som \xc3\xa9 muito estranho\nMuito obrigado, vai ajudar muito\nBruno, uma d\xc3\xbavida, tem algum site que guarda a informa\xc3\xa7\xc3\xa3o de estoque das vacinas por cidade?\n\xc3\x89 que eles erraram o endere\xc3\xa7o, acharam que era a Av. 9 de julho\n\xc3\x89 sigilo de ce

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(182119,), dtype=int64, numpy=array([50, 61, 81, ..., 78, 61,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

S
a
u
d
a
d
e
s
 
d


In [16]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'S' b'a' b'u' b'd' b'a' b'd' b'e' b's' b' ' b'd' b'e' b'm' b'a' b'i'
 b's' b' ' b'g' b'a' b't' b'\xc3\xa3' b'o' b'\n' b'S' b'a' b'u' b'd' b'a'
 b'd' b'e' b's' b' ' b'd' b'o' b'n' b'a' b' ' b'E' b'r' b'c' b'i' b'l'
 b'h' b'a' b'\n' b'M' b'e' b'u' b's' b' ' b'p' b'\xc3\xaa' b's' b'a' b'm'
 b'e' b's' b' ' b'L' b'u' b',' b' ' b'n' b'\xc3\xa3' b'o' b' ' b'e' b'x'
 b'i' b's' b't' b'e' b'm' b' ' b'p' b'a' b'l' b'a' b'v' b'r' b'a' b's'
 b' ' b'p' b'a' b'r' b'a' b' ' b'c' b'o' b'n' b'f' b'o' b'r' b't' b'a'
 b'r' b' ' b'n' b'e' b's' b't'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Saudades demais gat\xc3\xa3o\nSaudades dona Ercilha\nMeus p\xc3\xaasames Lu, n\xc3\xa3o existem palavras para confortar nest'
b'e momento, mas se precisar conversar, estou por aqui\n_ Autoestima abaixou na pandemia\nParab\xc3\xa9ns Erika!'
b'\nAgora j\xc3\xa1 to me achando hahahaha obrigado por fazer meu dia Natinha\nEu queria ter pelo menos 1% do ch'
b'arme do \nAcabamos de abrir vagas para QA aqui no QuintoAndar, que tal vir revolucionar o mercado imob'
b'ili\xc3\xa1rio conosco? Estamos com grandes desafios na \xc3\xa1rea, com muito espa\xc3\xa7o para inova\xc3\xa7\xc3\xa3o e crescimento. '


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Saudades demais gat\xc3\xa3o\nSaudades dona Ercilha\nMeus p\xc3\xaasames Lu, n\xc3\xa3o existem palavras para confortar nes'
Target: b'audades demais gat\xc3\xa3o\nSaudades dona Ercilha\nMeus p\xc3\xaasames Lu, n\xc3\xa3o existem palavras para confortar nest'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [24]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 126) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  32256     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  129150    
Total params: 4,099,710
Trainable params: 4,099,710
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [30]:
sampled_indices

array([ 46,  45,  18,  20,  91,  31,  29,  34,  21,  11,  17, 118,  84,
        81, 100,  15, 123,   0, 115,  12,  99, 106,  38,  29,  88, 112,
        66, 104,  39,  34,  66,  23,  85,  78, 125,   6,  48,   6,  53,
         3,  21,   9,   9,  67,  66, 102,  14, 115,  68,  59, 106,  99,
       114,  82,  61,  76,  21,  35,  79, 102,  93, 104, 125,  74,  21,
       104, 121,   8, 102,  61,  91,  73,  79,  91,   8,  32,  68,  71,
         0, 117,  18,  83,  60,  93,  85,  79,  73, 107,  43,  80,  91,
       125,  84,  75,  13,  90,  69, 106,  25, 125])

Decode these to see the text predicted by this untrained model:

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'minha internet\nPessoas que j\xc3\xa1 tem dinheiro ou que ter\xc3\xa3o in\xc3\xbameras chances na televis\xc3\xa3o - Americanas: '

Next Char Predictions:
 b"ON13\xc2\xb0@=C4*0\xe2\x83\xa3xu\xc3\x93.\xe7\x94\xbb[UNK]\xe2\x80\x8b+\xc3\x8d\xc3\xa7G=}\xc3\xb4f\xc3\xa2HCf6yr\xf0\x9f\xa5\xb2%Q%V!4((gf\xc3\xa0-\xe2\x80\x8bh]\xc3\xa7\xc3\x8d\xc3\xbavap4Ds\xc3\xa0\xc3\x81\xc3\xa2\xf0\x9f\xa5\xb2n4\xc3\xa2\xe3\x83\xbc'\xc3\xa0a\xc2\xb0ms\xc2\xb0'Ahk[UNK]\xe2\x80\xa61w_\xc3\x81ysm\xc3\xa9Lt\xc2\xb0\xf0\x9f\xa5\xb2xo,\xc2\xa1i\xc3\xa78\xf0\x9f\xa5\xb2"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 126)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.835746


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [34]:
tf.exp(mean_loss).numpy()

125.93247

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [35]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [37]:
EPOCHS = 80

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/80
28/28 [==============================] - 4s 58ms/step - loss: 3.8954
Epoch 2/80
28/28 [==============================] - 2s 57ms/step - loss: 2.9901
Epoch 3/80
28/28 [==============================] - 2s 57ms/step - loss: 2.6616
Epoch 4/80
28/28 [==============================] - 2s 58ms/step - loss: 2.4982
Epoch 5/80
28/28 [==============================] - 2s 57ms/step - loss: 2.4024
Epoch 6/80
28/28 [==============================] - 2s 58ms/step - loss: 2.3386
Epoch 7/80
28/28 [==============================] - 2s 59ms/step - loss: 2.2851
Epoch 8/80
28/28 [==============================] - 2s 58ms/step - loss: 2.2313
Epoch 9/80
28/28 [==============================] - 2s 58ms/step - loss: 2.1731
Epoch 10/80
28/28 [==============================] - 2s 59ms/step - loss: 2.1118
Epoch 11/80
28/28 [==============================] - 2s 59ms/step - loss: 2.0492
Epoch 12/80
28/28 [==============================] - 2s 60ms/step - loss: 1.9816
Epoch 13/80
28/28 [==================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [41]:
start = time.time()
states = None
next_char = tf.constant(['Bolsonaro'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Bolsonaro 
Precisamos aderir essa campanha no Brasil e repretendo de recebendo notificad dela eu deixo and para o coisas que poderiam ter acontecido dentro do BBB 
Eu quero acreditar que o Fiuk tem chul
Vi Fayacassim, Valeu migo, to começando a ter isso para me ajudar a processar e quando loucade, já emo deciscro, eu só quero Ivy ou Flay eliminada pra dormir em pazy
É claro minha jogador é boma pelo corpar todo de amigo? 
_ Brani eu tô me dá viga, investidação de modelo com a música
Envindo vai ter tendo chances internet e rapaz vou ter bastante tempo pra montar hahahahahaha 
Mas Xenneg eu bico na proprecida do meu pai, prociso e estudo em isso é daquera dova então . Estou das poderiam retirar o pandeiro e ficando com fome
Alguém sabe sei que eu preciso mesmo, mas fico a disposição para qualquer conversa
Acho que eu me do ou planetar um que de quebra Chapada dos Guimarães, Mato Grosof energuns do mundo interporte &lt;3 espero que a dona Netflix no Teamir o tema féria muito pra vc e mar

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [42]:
start = time.time()
states = None
next_char = tf.constant(['Bolsonaro', 'Bolsonaro', 'Bolsonaro', 'Bolsonaro', 'Bolsonaro'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Bolsonaro \nProntinho, voc\xc3\xaa j\xc3\xa1 deve conseguir ver o resultado no site agora, qualquer d\xc3\xbavida \xc3\xa9 s\xc3\xb3 me chamar \nProntinho, voc\xc3\xaa j\xc3\xa1 deve conseguir ver o resultado no site agora, qualquer d\xc3\xbavida \xc3\xa9 s\xc3\xb3 me chamar \nProntinho, voc\xc3\xaa j\xc3\xa1 deve conseguir ver o resultado no site agora, qualquer d\xc3\xbavida \xc3\xa9 s\xc3\xb3 me chamar \nOie, vou verificar o que aconteceu e te aviso quando estiver funcionando\nResolvido, s\xc3\xb3 acessar o site :D  quando terminar a quarentena memendo sobre isso o Mais o nome do min todo mundo descunsidar com a mat\xc3\xa9ria-prima usada, d\xc3\xa1 at\xc3\xa9 uma dor\xc3\xa7aria melhor com um reencontro da galera tomando um parquinho"0 Acho que s\xc3\xb3 acessar o site :D  quart obbeliades de preocuadar de novos casos, n\xc3\xb3s tivemos uma ascens\xc3\xa3o acentuada no tempo e de arrigando o \xc3\xbaltimo epis\xc3\xb3dio de Modern Family, gente to chorando horrores, 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [43]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [44]:
states = None
next_char = tf.constant(['Bolsonaro'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Bolsonaro 
Precisamos aderir essa campanha no Brasil, gostaria muito do seu feedback 
Olhando story do Sinte 


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [47]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [48]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [49]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [50]:
model.fit(dataset, epochs=1)

28/28 [==============================] - 4s 60ms/step - loss: 3.9375


Or if you need more control, you can write your own complete custom training loop:

In [51]:
EPOCHS = 62

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 3.1688

Epoch 1 Loss: 2.9881
Time taken for 1 epoch 2.33 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.7750

Epoch 2 Loss: 2.6534
Time taken for 1 epoch 1.74 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.5583

Epoch 3 Loss: 2.4904
Time taken for 1 epoch 1.76 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 2.4237

Epoch 4 Loss: 2.3943
Time taken for 1 epoch 1.78 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 2.3294

Epoch 5 Loss: 2.3307
Time taken for 1 epoch 1.87 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 2.3211

Epoch 6 Loss: 2.2750
Time taken for 1 epoch 1.80 sec
________________________________________________________________________________
Epoch 7 Batch 0 Loss 2

In [55]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant(['Papai'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Papai na força falando de um dia para o tio Crizer alguma coisa no banho e eu não enxergar hahahha pasar que eu tbm só faço cagada, essa mesm? Se for sou yuth amor de Baladork da Queria estar momento #BBB21
Saudades dona Ercilha
Meus pêsames Lu, não existem palavras para chance boa anjesicê como eu vim parar aqui? Eu só tenho 6 anos" 
Se não tinha contado pro @_lucaspg_ que o Dexametasona o Babl #BBB20
Hahahaha, amei a gente tina aqui hahahahaha
Panq mais às proteidos hahahahaha, saudades Renanzudo
Socorro, vou muito as brintes de assistir estão pormina de casasão! 
Já perdimo a gente quer que caia: Bolsonaro Quembécide, iscoluções e persoal durando o google apineis do céteito nes deixar o qua lome a Clarreada né?
Ao gente enteu escrever uma thread sobre todos os momentos inclusivos e didáticos, todo mundo deveria assistir
Se não tiver gritter não sei se tinha am que poderia ver se dar dia de olhar nesolinhos, de um filho , fica o live tão buirra, tudo menos tom uma from t. Acho que é 